[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/cats-and-dogs-classification/blob/master/cats_and_dogs_classificatoin.ipynb)

In [0]:
%%capture
!pip install kaggle

from zipfile import ZipFile
import io, cv2, fnmatch, shutil, os, getpass, subprocess, random

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from time import time
from glob import glob
from sklearn.utils import class_weight
from google.colab import files

In [0]:
root_dir = '/content'
download_dir = os.path.join(root_dir,'cats_dogs')
data_dir = os.path.join(download_dir,'data')
train_path = os.path.join(data_dir,'train')
val_path = os.path.join(data_dir,'val')
train_dogs_path = os.path.join(train_path,'dogs')
train_cats_path = os.path.join(train_path,'cats')
val_dogs_path = os.path.join(val_path,'dogs')
val_cats_path = os.path.join(val_path,'cats')

data source - https://www.kaggle.com/c/dogs-vs-cats/data

In [10]:
os.environ['KAGGLE_CONFIG_DIR'] = root_dir
os.chdir(root_dir)
if 'kaggle.json' not in os.listdir(root_dir):downloaded = files.upload()
if 'cats_dogs' in os.listdir(root_dir):shutil.rmtree(download_dir)
os.mkdir(download_dir)
os.chdir(download_dir)
!kaggle competitions download -c dogs-vs-cats
!unzip -q -o train.zip
!unzip -q -o test1.zip

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 38.3MB/s]
 98% 267M/271M [00:03<00:00, 116MB/s] 
100% 271M/271M [00:03<00:00, 91.0MB/s]
 96% 521M/543M [00:07<00:00, 92.7MB/s]
100% 543M/543M [00:07<00:00, 72.0MB/s]


In [11]:
cat_pattern = '*cat.*.jpg'
dog_pattern = '*dog.*.jpg'

images = glob('/content/cats_dogs/train/*.jpg', recursive=True)
cats = fnmatch.filter(images,cat_pattern)
dogs = fnmatch.filter(images,dog_pattern)

if 'data' not in os.listdir(download_dir):os.mkdir(data_dir)
if 'train' not in os.listdir(data_dir):os.mkdir(train_path)
if 'dogs' not in os.listdir(train_path):os.mkdir(train_dogs_path)
if 'cats' not in os.listdir(train_path):os.mkdir(train_cats_path)
  
if 'val' not in os.listdir(data_dir):os.mkdir(val_path)
if 'dogs' not in os.listdir(val_path):os.mkdir(val_dogs_path)
if 'cats' not in os.listdir(val_path):os.mkdir(val_cats_path)

for file in cats: shutil.copy2(file, train_cats_path)
for file in dogs: shutil.copy2(file, train_dogs_path)
  
  
# split train date into train and validation
train_len = len(os.listdir(train_dogs_path))
val_len = train_len * 0.3
val_dogs = random.sample(os.listdir(train_dogs_path),int(val_len))
val_cats = random.sample(os.listdir(train_cats_path),int(val_len))

for file in val_dogs:
  try: shutil.move(os.path.join(train_dogs_path,file), val_dogs_path)
  except: pass
for file in val_cats:
  try: shutil.move(os.path.join(train_cats_path,file), val_cats_path)
  except: pass
  
print(len(os.listdir(train_cats_path)))
print(len(os.listdir(val_cats_path)))

print(len(os.listdir(train_dogs_path)))
print(len(os.listdir(val_dogs_path)))

print('total train samples ', len(os.listdir(train_cats_path)) + len(os.listdir(train_dogs_path)))
print('total train samples ', len(os.listdir(val_cats_path)) + len(os.listdir(val_dogs_path)))

8750
3750
8750
3750
total train samples  17500
total train samples  7500


#CNN classifier

In [68]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size=64
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=( 150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        #steps_per_epoch=18631 // batch_size,
        epochs=16,
        validation_data=validation_generator,
        #validation_steps=10119 // batch_size
        )
model.save_weights('first_try.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
Epoch 1/16
274/274 [==============================] - 185s 674ms/step - loss: 0.6532 - acc: 0.6105 - val_loss: 0.5945 - val_acc: 0.6825
Epoch 2/16
274/274 [==============================] - 176s 644ms/step - loss: 0.5644 - acc: 0.7116 - val_loss: 0.4892 - val_acc: 0.7596
Epoch 3/16
274/274 [==============================] - 175s 639ms/step - loss: 0.5034 - acc: 0.7541 - val_loss: 0.5124 - val_acc: 0.7473
Epoch 4/16
274/274 [==============================] - 177s 644ms/step - loss: 0.4489 - acc: 0.7907 - val_loss: 0.3812 - val_acc: 0.8313
Epoch 5/16
274/274 [==============================] - 176s 642ms/step - loss: 0.4105 - acc: 0.8101 - val_loss: 0.3582 - val_acc: 0.8409
Epoch 6/16
274/274 [==============================] - 177s 647ms/step - loss: 0.3770 - acc: 0.8320 - val_loss: 0.3582 - val_acc: 0.8429
Epoch 7/16
274/274 [==============================] - 177s 647ms/step - loss: 0.3444 - acc: 0.8483 

#Transfer learning

In [0]:
#BASE_MODEL = 'VGG16'
from keras.preprocessing.image import ImageDataGenerator

def transfer_learning(BASE_MODEL):
  if BASE_MODEL=='VGG16':
      from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
  elif BASE_MODEL=='vgg19':
      from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
  elif BASE_MODEL=='ResNet50':
      from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
  elif BASE_MODEL=='InceptionV3':
      from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
  elif BASE_MODEL=='Xception':
      from keras.applications.xception import Xception as PTModel, preprocess_input
  elif BASE_MODEL=='DenseNet169': 
      from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
  elif BASE_MODEL=='DenseNet121':
      from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
  else:
      raise ValueError('Unknown model: {}'.format(BASE_MODEL))
  
  import keras
  keras.backend.set_learning_phase(1)
  
  check_point_name = BASE_MODEL + '.model'
  model_weights = BASE_MODEL + '.h5'

  train_datagen = ImageDataGenerator(
          rescale=1./255,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          preprocessing_function = preprocess_input)

  val_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function = preprocess_input)

  batch_size=32
  train_generator = train_datagen.flow_from_directory(
          '/content/cats_dogs/data/train',  # this is the target directory
          target_size=(250, 250),  # all images will be resized to 150x150
          batch_size=batch_size,
          class_mode='binary') 

  validation_generator = val_datagen.flow_from_directory(
          '/content/cats_dogs/data/val',
          target_size=(250, 250),
          batch_size=batch_size,
          class_mode='binary')    

  img_rows, img_cols, img_channel = 250, 250, 3
  base_model = PTModel(weights='imagenet'
                     ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

  add_model = Sequential()
  add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
  add_model.add(Dense(64, activation='relu'))
  add_model.add(Dense(1, activation='sigmoid'))

  model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


  for layer in base_model.layers:
      layer.trainable = False

      if layer.name.startswith('bn'):
          layer.call(layer.input, training=False)


  model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
                metrics=['accuracy'])
  start_time = time()
  model.fit_generator(
          train_generator,
          epochs=16,
          validation_data=validation_generator,
          #class_weight = class_weights,
          callbacks=[ModelCheckpoint(check_point_name, monitor='val_acc', save_best_only=True)])
  model.save_weights(model_weights)

  print('time taken ',time()-start_time)

In [73]:
transfer_learning('VGG16')

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
58892288/58889256 [==============================] - 1s 0us/step
Epoch 1/16
547/547 [==============================] - 330s 604ms/step - loss: 0.4231 - acc: 0.8201 - val_loss: 0.2916 - val_acc: 0.8900
Epoch 2/16
547/547 [==============================] - 322s 588ms/step - loss: 0.2869 - acc: 0.8872 - val_loss: 0.2411 - val_acc: 0.9068
Epoch 3/16
547/547 [==============================] - 322s 588ms/step - loss: 0.2473 - acc: 0.9021 - val_loss: 0.2187 - val_acc: 0.9136
Epoch 4/16
547/547 [==============================] - 321s 588ms/step - loss: 0.2302 - acc: 0.9078 - val_loss: 0.2067 - val_acc: 0.9151
Epoch 5/16
547/547 [==============================] - 321s 587ms/step - loss: 0.2163 - acc: 0.9111 - val_loss: 0.2091 - val_acc: 0.9135
Epoch 6/16
547/547 [==============================] - 321s 586ms/step - loss: 0.2038 - acc: 0.9180 - val_loss: 0.1885 - val_acc: 0.9231
Epoch 7/16
547/547 [==============

In [30]:
transfer_learning('DenseNet169')

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
Epoch 1/16
547/547 [==============================] - 470s 860ms/step - loss: 0.0795 - acc: 0.9693 - val_loss: 0.0483 - val_acc: 0.9811
Epoch 2/16
547/547 [==============================] - 449s 821ms/step - loss: 0.0461 - acc: 0.9817 - val_loss: 0.0443 - val_acc: 0.9831
Epoch 3/16
547/547 [==============================] - 451s 825ms/step - loss: 0.0352 - acc: 0.9870 - val_loss: 0.0404 - val_acc: 0.9851
Epoch 4/16
547/547 [==============================] - 450s 823ms/step - loss: 0.0322 - acc: 0.9872 - val_loss: 0.0399 - val_acc: 0.9841
Epoch 5/16
547/547 [==============================] - 450s 822ms/step - loss: 0.0277 - acc: 0.9901 - val_loss: 0.0429 - val_acc: 0.9848
Epoch 6/16
547/547 [==============================] - 452s 826ms/step - loss: 0.0253 - acc: 0.9910 - val_loss: 0.0379 - val_acc: 0.9853
Epoch 7/16
547/547 [==============================] - 451s 824ms/step - loss: 0.0218 - acc: 0.9920 

In [0]:
# running on TPU

# !pip install keras==2.1
# import tensorflow as tf
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense


BASE_MODEL = 'VGG16'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(loss='binary_crossentropy'
              ,optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator
        #class_weight = class_weights,
        ,callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)]
        )
model.save_weights('Xception.h5')

print('time taken ',time()-start_time)